# Ranking Strategies Notebook
This notebook normalized cumulative, reliability, and decay metrics and ranks strategies highest to lowest based on these metrics.

## Import Libraries

In [1]:
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import  util
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from EquityHedging.analytics import hedge_metrics as hm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
    


## Import returns data

### Imports Daily, Weekly, Monthly, Quarterly and Yearly returns data

To import the returns data:
* Select an Equity Benchmark (**equity_bmk**) - SPTR, M1WD, SX5T
* Decide if you want a Fixed Income Benchmark (**include_fi**) - True, False
* Select what equity hedging strategies to exclude (**strat_drop_list**). Below is the list of strategies:

    * 99%/90% Put Spread, Down Var, Vortex, VOLA , Dynamic Put Spread, VRR, GW Dispersion, Corr Hedge

In [2]:
equity_bmk = 'M1WD'
include_fi = False
weighted = [True, False]
strat_drop_list = ['99%/90% Put Spread', 'Vortex']
new_strat = True
returns = dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

##  Get Notional Weights 

In [3]:
#notional_weights = dm.get_notional_weights(returns['Monthly'])
notional_weights = [19.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0]
df_weights = get_df_weights(notional_weights, list(returns['Monthly'].columns), include_fi)
fmt.get_notional_styler(df_weights)

,M1WD,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var
Notional Weights (Billions),$19.00,$1.00,$1.00,$1.00,$1.00,$1.00,$0.25,$1.00
Percentage Weights,100.00%,5.26%,5.26%,5.26%,5.26%,5.26%,1.32%,5.26%
Strategy Weights,0.00%,16.00%,16.00%,16.00%,16.00%,16.00%,4.00%,16.00%


## Get Normalized Data

In [4]:
df_normal= summary.get_normalized_hedge_metrics(returns, equity_bmk, notional_weights, weighted_hedge = True)
df_normal


,Down Var,VOLA,Dynamic Put Spread,VRR,GW Dispersion,Corr Hedge,Def Var,Weighted Hedges
Benefit,0.447231,0.000000,0.155304,1.000000,0.243241,0.374030,0.422692,0.007034
Down Reliability,0.535432,0.000000,0.154297,0.972523,0.401138,0.872183,1.000000,0.111236
Up Reliability,1.000000,0.257301,0.000000,0.699599,0.746761,0.856815,0.618485,0.567331
Convexity,0.710750,1.000000,0.035714,0.310656,0.119773,0.844444,0.000000,0.186045
Cost,0.086945,0.302052,0.535959,0.000000,0.668714,0.569583,1.000000,0.919781
Decay,0.337719,0.828947,0.293860,0.000000,1.000000,0.877193,0.767544,0.951754


## Creates a List of Symbols 

In [5]:
#get length of columns
n=len(df_normal.columns)

#get column names of df_normal
c=list(df_normal.columns)

#creates a list of the corresponding number of the symbols we want
a=list(range(0,33))
b=list(range(46,53))
for i in b:
    a.append(i)
    
#Creates a data frame that assigns each strategy with a symbol
symbols=pd.DataFrame(columns=c,index=[0])
symbols.iloc[0] = a[0:n]


## Graph Normalized Data and Ranking Each Strategy

In [11]:
index = list(df_normal.index)

fig = go.Figure()
#loops through eaach column and plots each metric for the given strategy
for col in df_normal.columns:
    fig.add_trace(go.Scatter(
        x=index,
        y=df_normal[col],
        marker=dict(size = 20),
        marker_symbol = symbols[col],
        mode= "markers",
        name=col
        ))
fig.update_layout(title="",
                xaxis_title="Hedge Metric",
                yaxis_title="Higher the Better")
fig.show()